# Lab4: Semántica léxica (Lexical Semantic)

#### Accediendo a WordNet

In [ ]:
from nltk.book import *
from nltk.corpus import wordnet as wn

In [2]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [3]:
wn.synsets('dog', pos=wn.VERB)

[Synset('chase.v.01')]

In [4]:
wn.synset('dog.n.01')

Synset('dog.n.01')

In [5]:
wn.synset('dog.n.02')

Synset('frump.n.01')

In [6]:
wn.synset('frump.n.01')

Synset('frump.n.01')

In [7]:
wn.synset('dog.n.01').definition()

'a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds'

In [8]:
wn.synset('dog.n.02').definition()

'a dull unattractive unpleasant girl or woman'

In [9]:
wn.synset('dog.n.01').lemmas()

[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

In [10]:
wn.synset('dog.n.02').lemmas()

[Lemma('frump.n.01.frump'), Lemma('frump.n.01.dog')]

#### Relaciones entre palabras con WordNet

In [11]:
dog = wn.synset('dog.n.01')
dog.hypernyms()

[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

In [12]:
dog.hyponyms()

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

In [13]:
good = wn.synset('good.a.01')
good.lemmas()

[Lemma('good.a.01.good')]

In [14]:
g0 = good.lemmas()[0]
g0.antonyms()

[Lemma('bad.a.01.bad')]

In [15]:
dog.part_meronyms()

[Synset('flag.n.07')]

In [16]:
dog.member_meronyms()

[]

In [17]:
dog.part_holonyms()

[]

In [18]:
dog.member_holonyms()

[Synset('canis.n.01'), Synset('pack.n.06')]

##### Ejercicio: imprimir las definiciones de flag.n.07, canis.n.01 y pack.n.06 y ver porque estos synsets estan realcionados de esta manera

In [19]:
print(wn.synset('flag.n.07').definition())
print(wn.synset('canis.n.01').definition())
print(wn.synset('pack.n.06').definition())

a conspicuously marked or shaped tail
type genus of the Canidae: domestic and wild dogs; wolves; jackals
a group of hunting animals


**Flag** se relaciona con una cola, que es una parte del perro, por lo que es un merónimo de perro.
**Canis** es un todo del cuál perro es parte, ya que se refiere a perros domésticos y salvajes, por lo que es un holónimo de perro.
**Pack** también es un todo del cuál perro es parte, ya que se refiere a grupo de animales de caza, por lo que también es un holónimo de perro

#### Desambiguación de palabras por sentido

##### Ejercicio: ¿Cuántos sentidos sustantivos de "banco" hay en WordNet?

“The bank can guarantee deposits will eventually cover future tuition costs because it invests in adjustable-rate mortgage securities.”

In [20]:
synsets_bank = wn.synsets('bank', pos=wn.NOUN)
print(f"Hay un total de {len(synsets_bank)} sentidos sustantivos de \"banco\" en Wordnet")
for synset in synsets_bank:
    print(f"{synset} : {synset.definition()}")

Hay un total de 10 sentidos sustantivos de "banco" en Wordnet
Synset('bank.n.01') : sloping land (especially the slope beside a body of water)
Synset('depository_financial_institution.n.01') : a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.03') : a long ridge or pile
Synset('bank.n.04') : an arrangement of similar objects in a row or in tiers
Synset('bank.n.05') : a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.06') : the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.07') : a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('savings_bank.n.02') : a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.09') : a building in which the business of banking transacted
Synset('bank.n.10') : a flight maneuver; aircraft tips laterally about 

El sentido correcto de la palabra **banco** en la oración anterior es **depository_financial_institution.n.01**

Ahora veamos que nos da el algoritmo de Lesk:

In [21]:
from nltk.wsd import lesk
from nltk import word_tokenize
S = "The bank can guarantee deposits will eventually cover future tuition costs \
because it invests in adjustable- rate mortgage securities."
S_tok = word_tokenize(S)
print(lesk(S_tok,'bank','n'))

Synset('bank.n.05')


El resultado producido por Lesk no fue el esperado. Hagamos algunos diagnósticos:

In [22]:
l = word_tokenize((wn.synset('bank.n.05').definition()))
m = word_tokenize((wn.synset('bank.n.02').definition()))
k = set(S_tok)
k.intersection(l)

{'future', 'in'}

In [23]:
k.intersection(m)

{'deposits'}

Analizando el [código fuente](http://www.nltk.org/_modules/nltk/wsd.html) de cómo esta implementado Lesk vemos que el algoritmo realiza una intersección entre la oración a analizar y las definiciones de los sentidos de _banco_, y da como resultado el sentido cuya intersección sea la que mayor cantidad de elementos tenga

Por eso es que en los diagnósticos que hicimos vemos que la cantidad de elementos de la intersección con _bank.n.05_ es mayor a la cantidad de elementos de la intersección con el _bank.n.02_. Es por eso que _Lesk_ nos da un resultado erróneo para este caso

**Cambios que sugeriría a NLTK:** sugeriría tanto para la oración como para las definiciones de los synsets remover stopwords y realizar stemming, eso como principal. Otra cosa interesante que podría aplicarse es algún algoritmo como el de Similitud de Coseno con la oración sin stopwords y stemeada con cada definición, y quedarnos con el synset que más porcentaje de similitud nos de

##### Ejercicio: para cada ejemplo, a continuación descubra el sentido correcto de WordNet según su criterio. Puede usar la [búsqueda en línea de WordNet](http://wordnetweb.princeton.edu/perl/webwn) para navegar a través de los sentidos

In [24]:
oracion_1 = "I went to the bank to deposit some money."
oracion_2 = "She created a big mess of the birthday cake."
oracion_3 = "In the interest of your safety, please wear your seatbelt."
oracion_4 = "I drank some ice cold water."

In [33]:
synset = lesk(word_tokenize(oracion_1),'bank', 'n')
print(f"Resultado de Lesk = {synset}: {synset.definition()}")
print(f"Pero debería haber sido: {wn.synset('bank.n.02')}: {wn.synset('bank.n.02').definition()}")

Resultado de Lesk = Synset('savings_bank.n.02'): a container (usually with a slot in the top) for keeping money at home
Pero debería haber sido: Synset('depository_financial_institution.n.01'): a financial institution that accepts deposits and channels the money into lending activities


In [34]:
synset = lesk(word_tokenize(oracion_2),'mess', 'n')
print(f"Resultado de Lesk = {synset}: {synset.definition()}")
print(f"Pero debería haber sido: {wn.synset('mess.n.01')}: {wn.synset('mess.n.01').definition()}")

Resultado de Lesk = Synset('mess.n.04'): a meal eaten in a mess hall by service personnel
Pero debería haber sido: Synset('mess.n.01'): a state of confusion and disorderliness


In [35]:
synset = lesk(word_tokenize(oracion_3),'interest', 'n')
print(f"Resultado de Lesk = {synset}: {synset.definition()}")
print(f"Pero debería haber sido: {wn.synset('interest.n.02')}: {wn.synset('interest.n.02').definition()}")

Resultado de Lesk = Synset('interest.n.04'): a fixed charge for borrowing money; usually a percentage of the amount borrowed
Pero debería haber sido: Synset('sake.n.01'): a reason for wanting something done


In [37]:
synset = lesk(word_tokenize(oracion_4),'ice', 'n')
print(f"Resultado de Lesk = {synset}: {synset.definition()}")
print(f"Pero debería haber sido: {wn.synset('ice.n.02')}: {wn.synset('ice.n.02').definition()}")

Resultado de Lesk = Synset('ice_rink.n.01'): a rink with a floor of ice for ice hockey or ice skating
Pero debería haber sido: Synset('ice.n.02'): the frozen part of a body of water


#### Un poco más allá

In [38]:
from nltk.stem import WordNetLemmatizer
wnLemmatizer = WordNetLemmatizer()
wnLemmatizer.lemmatize('dogs', 'n')

'dog'

In [39]:
wnLemmatizer.lemmatize('walking','v')

'walk'

In [40]:
wnLemmatizer.lemmatize('dogs')

'dog'

##### Ejercicio: ¿Podrías mejorar el algoritmo de Lesk con un lematizador? Si es así, ¿podrías escribir una función de match, que tome dos cadenas y devuelva las palabras coincidentes entre ellas? Queremos que las coincidencias (matches) sean útiles para un algoritmo como Lesk.¿Quizás también necesites usar un etiquetador POS? Podes usar un etiquetador POS incorporado en WordNet

In [102]:
from nltk.corpus import stopwords
from nltk import pos_tag
import re

def get_tag_to_lemmatize(tag):
    v_or_n = 'v' if str(tag).startswith('VB') else 'n'
    return v_or_n
    
def get_lemmatized_string(string):
    sw = stopwords.words('english')
    string_without_sw = [word for word in word_tokenize(string) if re.search("\w", word) and word not in sw]
    return set([wnLemmatizer.lemmatize(word, get_tag_to_lemmatize(tag)) for (word, tag) in pos_tag(string_without_sw)])

def match(string, another_string):
    lemmatized_string = get_lemmatized_string(string)
    another_lemmatized_string = get_lemmatized_string(another_string)
    return lemmatized_string.intersection(another_lemmatized_string)

Ahora probemos nuestra función match sobre la oracion1 del ejercicio anterior y el resultado que nos habia dado Lesk

In [103]:
match(wn.synset('savings_bank.n.02').definition(), oracion_1)

{'money'}

Lo mismo pero con el synset que nos debería haber dado Lesk

In [104]:
match(wn.synset('bank.n.02').definition(), oracion_1)

{'deposit', 'money'}

Con este resultado podemos ver que si implementaramos nuestra propia función **_Lesk_** usando **_match_**, teniendo en cuenta que Lesk se queda con el synset que más elementos haya dado la intersección, ahora funcionaría correctamente para la palabra **_bank_** en la oración 1

Vamos a hacerlo, implementemos nuestra propia función **_Lesk_** usando **_match_**:

In [105]:
def my_lesk(context_sentence, ambiguous_word, pos=None, synsets=None):
    if synsets is None:
        synsets = wordnet.synsets(ambiguous_word)
    if pos:
        synsets = [ss for ss in synsets if str(ss.pos()) == pos]
    if not synsets:
        return None
    _, sense = max((len(match(context_sentence, ss.definition())), ss) for ss in synsets)
    return sense

Ahora que ya tenemos nuestra función **_Lesk_**, la corremos sobre los 4 ejemplos del ejercicio anterior:

In [110]:
synset = my_lesk(oracion_1, 'bank', 'n')
print(f"Resultado de nuestra función Lesk para la oracion1:\n{synset}: {synset.definition()}")

Resultado de nuestra función Lesk para la oracion1:
Synset('depository_financial_institution.n.01'): a financial institution that accepts deposits and channels the money into lending activities


In [111]:
synset = my_lesk(oracion_2, 'mess', 'n')
print(f"Resultado de nuestra función Lesk para la oracion2:\n{synset}: {synset.definition()}")

Resultado de nuestra función Lesk para la oracion2:
Synset('mess.n.04'): a meal eaten in a mess hall by service personnel


In [112]:
synset = my_lesk(oracion_3, 'interest', 'n')
print(f"Resultado de nuestra función Lesk para la oracion3:\n{synset}: {synset.definition()}")

Resultado de nuestra función Lesk para la oracion3:
Synset('sake.n.01'): a reason for wanting something done


In [113]:
synset = my_lesk(oracion_4, 'ice', 'n')
print(f"Resultado de nuestra función Lesk para la oracion4:\n{synset}: {synset.definition()}")

Resultado de nuestra función Lesk para la oracion4:
Synset('ice_rink.n.01'): a rink with a floor of ice for ice hockey or ice skating


Resultados:
* Con **_Lesk de NLTK_**: Para las 4 oraciones de ejemplo tuvimos resultados incorrectos
* Con **_Nuestra función MyLesk_**: En la oraciones 1 y 3 tuvimos resultados correctos mientras que la 2 y la 4 fueron incorrectos

Por lo que vemos claramente que aplicarle a lesk un lemmatizador hace que el algoritmo funcione de mejor manera. Sin embargo, que siga fallando para las oraciones 2 y 4 nos da la pauta de que nunca es suficiente, y se podría seguir mejorando, como todo en NLP